# ATS MDA workflow

This workflow is a step-by-step guide to archive ATS model data associated with a manuscript on ESS-DIVE (https://docs.ess-dive.lbl.gov/). Purpose of this guide is to standardize and automate the data archving process. 

A few key concepts in the notebook:

- `Simulation Directory`: where ATS runs (e.g., a directory at $SCRATCH on HPC)
- `Data Package Directory`: where you want to create the data archive (it can be anywhere on HPC or local machine)
- `rsync`: a file transfer command line tool (use `apt install rsync` or `brew install rsync` to install it on Linux or macOS, respectively)

|     | where to archive | extention examples |
| --- | --- | --- |
| Files related to the manuscript | ESS-DIVE | exo, xml, h5, nc, csv |
| Files not related to the manuscript | HPSS | all |

This command below is to sync all files with certain extentions in the simulation directory to the data package directory.

```
rsync -avzP --include=*/ --include='*.exo' --include='*.xml' --include='*.h5' --include='*.nc' --include='*.csv' --exclude=* --prune-empty-dirs ./<Simulation Directory>/ ./<Data Package Directory>/
```

This workflow creates the following files that are needed in the data archiving step:

* flmd.csv : File Level Metadata
> The file level metadata is a csv containing a list of all files in the data package and descriptions of what the files contain.

* dd.csv : Data Dictionary
> The data dictionary is a csv file that lists all column and row headers in the data package's tabular files.


In [ ]:
import os
import glob
import logging
import subprocess
import requests
from io import StringIO
from html.parser import HTMLParser
from pathlib import Path
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s: %(message)s', datefmt='%m/%d/%Y %H:%M:%S')

In [ ]:
write_new_csv = False
inplace = False
scratch_path = os.environ.get('SCRATCH')

In [ ]:
simulation_dir = Path(scratch_path + '/cscr/') # Define your simulation directory
if not os.path.exists(simulation_dir):
    logging.warning(f'Simulation directory <{simulation_dir}> does not exist.')
else:
    logging.info(f'Found simulation directory <{simulation_dir}>.')

In [ ]:
data_pkg_dir = Path(scratch_path + '/my_ATS_MDA/') # Define the destination directory for your ATS MDA
try:
    os.mkdir(data_pkg_dir)
    logging.info(f'Data package directory <{data_pkg_dir}> created.')
except FileExistsError:
    logging.info(f'Data package directory <{data_pkg_dir}> exists.')
    pass

# Make a copy of the data with certain extentions from `Simulation Directory` to `Data Package Directory`

In [ ]:
exts = ['exo', 'xml', 'csv', 'dat', 'txt', 'xmf', 'h5', 'out', 'nc', 'jpg', 'png', 'pdf'] # determine your own file types
exts_to_include = ' '.join([f"--include='*.{f}'" for f in exts])
exts_to_include

In [ ]:
try:
    subprocess.run("rsync -avzP --include=*/ "+exts_to_include+f" --include='slurm*' --exclude=* --prune-empty-dirs {simulation_dir}/ {data_pkg_dir}", shell=True)
except subprocess.CalledProcessError as e:
    print(f'Command failed with exit code {e.returncode}')

In [ ]:
# find periodic checkpoints in run0 and run1
run0_dir = subprocess.check_output([f'cd {data_pkg_dir}; find -name "*run0*"'], shell=True, encoding='utf-8').rstrip()
run1_dir = subprocess.check_output([f'cd {data_pkg_dir}; find -name "*run1*"'], shell=True, encoding='utf-8').rstrip()
run0_dir, run1_dir

In [ ]:
run0_checkpoints = subprocess.check_output([f'cd {data_pkg_dir}; cd {run0_dir}; find -name "checkpoint*.h5"'], shell=True, encoding='utf-8').split('\n')[1:-1]
run1_checkpoints = subprocess.check_output([f'cd {data_pkg_dir}; cd {run1_dir}; find -name "checkpoint*.h5"'], shell=True, encoding='utf-8').split('\n')[1:-1]
run0_checkpoints, run1_checkpoints

In [ ]:
run0_checkpoints_non_final, run1_checkpoints_non_final = [f for f in run0_checkpoints if 'final' not in f], [f for f in run1_checkpoints if 'final' not in f]
run0_checkpoints_non_final, run1_checkpoints_non_final

In [ ]:
# remove periodic checkpoints in run0 and run1
if len(run0_checkpoints_non_final) > 0:
    for f in run0_checkpoints_non_final:
        subprocess.run(f'cd {data_pkg_dir}; cd {run0_dir}; rm {f}', shell=True)
        logging.info(f'Removed <{data_pkg_dir}{run0_dir}{f}> from your MDA.')
else:
    logging.info('Did not find any non-final checkpoints in <run0>')
if len(run1_checkpoints_non_final) > 0:
    for f in run1_checkpoints_non_final:
        subprocess.run(f'cd {data_pkg_dir}; cd {run1_dir}; rm {f}', shell=True)
        logging.info(f'Removed <{data_pkg_dir}{run0_dir}{f}> from your MDA.')
else:
    logging.info('Did not find any non-final checkpoints in <run1>')

In [ ]:
run0_xmf = subprocess.check_output([f'cd {data_pkg_dir}; cd {run0_dir}; find -name "*.xmf"'], shell=True, encoding='utf-8').split('\n')[1:-1]
run1_xmf = subprocess.check_output([f'cd {data_pkg_dir}; cd {run1_dir}; find -name "*.xmf"'], shell=True, encoding='utf-8').split('\n')[1:-1]
run0_xmf, run1_xmf

In [ ]:
# remove xmf files in run0 and run1
if len(run0_xmf) > 0:
    for f in run0_xmf:
        subprocess.run(f'cd {data_pkg_dir}; cd {run0_dir}; rm {f}', shell=True)
        logging.info(f'Removed <{f}> from your MDA.')
else:
    logging.info('Did not find any xmf files in <run0>')
if len(run1_xmf) > 0:
    for f in run1_xmf:
        subprocess.run(f'cd {data_pkg_dir}; cd {run1_dir}; rm {f}', shell=True)
        logging.info(f'Removed <{f}> from your MDA.')
else:
    logging.info('Did not find any xmf files in <run1>')

In [ ]:
run0_vis = subprocess.check_output([f'cd {data_pkg_dir}; cd {run0_dir}; find -name "ats_vis_*.h5"'], shell=True, encoding='utf-8').split('\n')[1:-1]
run1_vis = subprocess.check_output([f'cd {data_pkg_dir}; cd {run1_dir}; find -name "ats_vis_*.h5"'], shell=True, encoding='utf-8').split('\n')[1:-1]
run0_vis, run1_vis

In [ ]:
# remove visualization files in run0 and run1
if len(run0_vis) > 0:
    for f in run0_vis:
        subprocess.run(f'cd {data_pkg_dir}; cd {run0_dir}; rm {f}', shell=True)
        logging.info(f'Removed <{f}> from your MDA.')
else:
    logging.info('Did not find any visualization files in <run0>')
if len(run1_vis) > 0:
    for f in run1_vis:
        subprocess.run(f'cd {data_pkg_dir}; cd {run1_dir}; rm {f}', shell=True)
        logging.info(f'Removed <{f}> from your MDA.')
else:
    logging.info('Did not find any visualization files in <run1>')

# Enumerate all files in this data package

In [ ]:
try:
    paths_and_files = sorted(subprocess.check_output([f'cd {data_pkg_dir}; find -name "*.*"'], shell=True, encoding='utf-8').split('\n')[1:-1])
    number_of_files = len(paths_and_files)
except subprocess.CalledProcessError as e:
    print(f'Command failed with exit code {e.returncode}')
number_of_files, paths_and_files

# Parse paths and filenames (needed in `flmd.csv`)

In [ ]:
files, paths = [], []
for path_and_file in paths_and_files:
    files.append(path_and_file.split('/')[-1])
    paths.append(path_and_file.replace(path_and_file.split('/')[-1], ''))

In [ ]:
files

In [ ]:
paths

In [ ]:
d = {'File_Name': files, 
     'File_Description': ['']*number_of_files, 
     'Standard': ['N/A']*number_of_files, 
     'Start_Date': [-9999]*number_of_files, 
     'End_Date': [-9999]*number_of_files, 
     'Missing_Value_Codes': ['N/A']*number_of_files, 
     'File_Path': paths}
d

In [ ]:
df = pd.DataFrame(data=d) 
df

In [ ]:
_dict = {'water_balance': 'Model output ascii file for the whole modeling domain.', 
         'checkpoint': 'Model output binary file as a checkpoint.',
         'xml': 'Model configuration file.',
         'xmf': 'Model Paraview visualization metadata file.',
         'slurm': 'Model screen printout file.',
         'pflotran': 'Model output file from the biogeochemical engine, Pflotran.',
         'ats_vis': 'Model output binary file with all time frames.',
         'LAI': 'Model input file (Leaf Area Index).',
         'exo': 'Model input file (compuational mesh).'}
_keys = list(_dict.keys())
for k in range(len(_dict)):
    df['File_Description'].iloc[[i if _keys[k] in df['File_Name'][i] else False for i in range(number_of_files)]] = _dict[_keys[k]]
df

In [ ]:
df.to_csv(data_pkg_dir/'flmd.csv')

# Find all csv files in this data package (needed in `dd.csv`)

In [ ]:
# Define your observation file and file format
obs_file_handle = 'water_balance'
obs_file_format = 'csv'

In [ ]:
csv_paths_and_files = subprocess.check_output([f'cd {data_pkg_dir}; find -name "{obs_file_handle}*.{obs_file_format}"'], shell=True, encoding='utf-8').split('\n')[1:-1]
print(csv_paths_and_files[:10], f'... AND {len(csv_paths_and_files)} MORE')

In [ ]:
for csv_file in csv_paths_and_files:
    with open(data_pkg_dir/csv_file) as f:
        lines = f.readlines()
        if csv_file == csv_paths_and_files[0]:
            print(lines[112])
            print()
            parameters = str([s.split(' [')[0] for s in lines[112].split(',')]).replace('\'','').replace('\"','').replace(', ',',').replace(' ','_')[1:-1].split(',')
            units = str([s.split(' [')[1] for s in lines[112].split(',')]).replace('\'','').replace('\"','').replace(', ',',').replace(']','')[1:-2].split(',')
            print(parameters)
            print()
            print(units)
    if write_new_csv and inplace: # caution! inplace is dangerous!
        with open(csv_file, 'w') as g:
            g.write(''.join(lines))
    elif write_new_csv:
        with open(csv_file+'_tmp', 'w') as g:
            g.write(''.join(lines))
number_of_parameters, number_of_units = len(parameters), len(units)

In [ ]:
logging.info(f'Numer of parameters: {number_of_parameters}')
logging.info(f'Numer of units: {number_of_units}')

In [ ]:
d = {'Column_or_Row_Name': parameters, 
     'Unit': units, 
     'Definition': ['']*number_of_parameters, 
     'Data_Type': ['numeric']*number_of_parameters, 
     'Term_Type': ['column_header']*number_of_parameters}

In [ ]:
df = pd.DataFrame(data=d) 
df

In [ ]:
# HTML approach (deprecated)

# class TableParser(HTMLParser):
#     def __init__(self):
#         super().__init__()
#         self.in_table = False
#         self.in_tr = False
#         self.in_th = False
#         self.in_td = False
#         self.headers = []
#         self.current_row = []
#         self.rows = []

#     def handle_starttag(self, tag, attrs):
#         if tag == "table":
#             self.in_table = True
#         elif tag == "tr" and self.in_table:
#             self.in_tr = True
#             self.current_row = []
#         elif tag == "th" and self.in_tr:
#             self.in_th = True
#         elif tag == "td" and self.in_tr:
#             self.in_td = True

#     def handle_endtag(self, tag):
#         if tag == "table":
#             self.in_table = False
#         elif tag == "tr":
#             if self.in_tr:
#                 if self.headers and len(self.current_row) == len(self.headers):
#                     self.rows.append(dict(zip(self.headers, self.current_row)))
#             self.in_tr = False
#         elif tag == "th":
#             self.in_th = False
#         elif tag == "td":
#             self.in_td = False

#     def handle_data(self, data):
#         if self.in_th:
#             self.headers.append(data.strip())
#         elif self.in_td:
#             self.current_row.append(data.strip())

In [ ]:
# ats_spec_url = "https://amanzi.github.io/ats/stable/input_spec/introduction.html"
# response = requests.get(ats_spec_url)
# parser = TableParser()
# parser.feed(response.text)

# result = {}
# for row in parser.rows:
#     if "Variable Root Name" in row and "Description" in row:
#         result[row["Variable Root Name"]] = row["Description"]

# logging.info(f'Numer of Variables found from ATS input spec: {len(result)}')
# result

In [ ]:
# ATS input spec symbol table approach

url = "https://raw.githubusercontent.com/amanzi/ats/refs/heads/master/docs/documentation/source/input_spec/symbol_table.org"
response = requests.get(url)
response.raise_for_status()
content = response.text

table_lines = [line for line in content.splitlines() if line.strip().startswith('|')]
table_text = "\n".join(table_lines)

df_input_spec = pd.read_csv(StringIO(table_text), sep='|', engine='python', skipinitialspace=True)
df_input_spec = df_input_spec.map(lambda x: x.strip() if isinstance(x, str) else x)
df_input_spec = df_input_spec.loc[:, df_input_spec.columns.str.strip().astype(bool)]

df_input_spec

In [ ]:
number_of_parameters_from_symbol_table = len(df_input_spec.iloc[1:, :])
logging.info(f'Numer of parameters from the symbol table in ATS input spec: {number_of_parameters_from_symbol_table}')

In [ ]:
_dict = dict(zip(df_input_spec.iloc[1:, 1], df_input_spec.iloc[1:, 3]))
_keys = list(_dict.keys())
_dict

In [ ]:
for k in range(len(_dict)):
    df['Definition'].iloc[[i if _keys[k] in df['Column_or_Row_Name'][i] else False for i in range(len(df))]] = _dict[_keys[k]]
df

In [ ]:
df.to_csv(data_pkg_dir/'dd.csv')

In [ ]:
# work in progress - read from xmf files

In [ ]:
# tree = ET.parse('/pscratch/sd/l/lizh142/onedrive/DESK/o/surs/nach_sur/ats_vis_surface_data.h5.80000.xmf').getroot()
# names = [attr.get("Name") for attr in tree.findall(".//Attribute")]
# print(names)